In [43]:
!pip install fastapi nest-asyncio pyngrok uvicorn aiofiles python-multipart

In [2]:
!mkdir templates

하위 디렉터리 또는 파일 templates이(가) 이미 있습니다.


In [3]:
!pip install aiofiles python-multipart

In [44]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import FileResponse
from typing import List
import os
import uvicorn

In [45]:
from fastapi import FastAPI, Request, Form
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
import nest_asyncio
from pyngrok import ngrok

import uvicorn
# from fastapi.staticfiles import StaticFiles

In [46]:
%%writefile templates/inputnews.html
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>뉴스 줄넘귀</title>
<style>
body {background-color: lightblue;}
.white {background-color: white;}
</style>
</head>

<body>

<h2>뉴스 줄넘귀</h2>

<form action="/topic" method = 'post'>
  <label for="keyword">검색할 키워드:</label><br>
  <input type="text" id="keyword" name="keyword" value=" "><br>
  <label for="ct">수집 뉴스의 수(숫자만 입력) :</label><br>
  <input type="int" id="ct" name="ct" value="3"><br><br>
  <input type="submit" value="Submit">
</form> 

<p>If you click the "Submit" button, the form-data will be sent to a page called "/action_page.php".</p>


</body>
</html>


Overwriting templates/inputnews.html


In [47]:
%%writefile templates/resultnews.html
<html>
<head>
<title>Topics</title>
</head>
<body>
    <h1>검색할 키워드: {{ keyword }} </h1>
    <h1>수집 뉴스의 수: {{ ct }}</h1>
    <h1>요약: {{ result_news }}</h1>
    <h2>뉴스</h2>
    #<audio src="./audioFinal.mp3" controls>
    <audio controls autoplay>
    <source src= {{ fn }} type="audio/mpeg">
        Your browser does not support the audio element.
    </audio>
</body>
</html>

Overwriting templates/resultnews.html


In [64]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, json, glob, time
from tqdm import tqdm

def process_a(keyword, ct):
    ###### 날짜 저장 ##########
    date = str(datetime.now())
    date = date[:date.rfind(':')].replace(' ', '_')
    date = date.replace(':','시') + '분'
    sleep_sec = 0.5

    ####### 언론사별 본문 위치 태그 파싱 함수 ###########
    print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)
    def crawling_main_text(url):
        req = requests.get(url)
        req.encoding = None
        soup = BeautifulSoup(req.text, 'html.parser')
        
        # 연합뉴스
        if ('://yna' in url) | ('app.yonhapnews' in url):
            main_article = soup.find('article', {'class':'story-news article'})
            if main_article == None:
                main_article = soup.find('div', {'class' : 'article-txt'})
        
            text = main_article.text
        
        # MBC 
        elif '//imnews.imbc' in url: 
            text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
        # 매일경제(미라클), req.encoding = None 설정 필요
        elif 'mirakle.mk' in url:
            text = soup.find('div', {'class' : 'view_txt'}).text
        
        # 매일경제, req.encoding = None 설정 필요
        elif 'mk.co' in url:
            text = soup.find('div', {'class' : 'art_txt'}).text
        
        # SBS
        elif 'news.sbs' in url:
            text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
        # KBS
        elif 'news.kbs' in url:
            text = soup.find('div', {'id' : 'cont_newstext'}).text
        
        # JTBC
        elif 'news.jtbc' in url:
            text = soup.find('div', {'class' : 'article_content'}).text
        
        # 그 외
        else:
            text == None
        
        return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
    press_nm = 'MBC'

    print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
   # query = input('검색할 키워드  : ')
   # news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

    print('\n' + '=' * 100 + '\n')

    print('브라우저를 실행시킵니다(자동 제어)\n')
    chrome_path = './chromedriver.exe'
    browser = webdriver.Chrome(chrome_path)

    news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(keyword)
    browser.get(news_url)
    time.sleep(sleep_sec)


######### 언론사 선택 및 confirm #####################
    print('설정한 언론사를 선택합니다.\n')

    search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
    search_opn_btn.click()
    time.sleep(sleep_sec)

    bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')
    time.sleep(sleep_sec)
# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
    press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
    press_tablist[1].click()
    time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
    bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

    press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
    press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


    for press_kind_btn in press_kind_btn_list:
    
        # 언론사 종류를 순차적으로 클릭(좌측)
        press_kind_btn.click()
        time.sleep(sleep_sec)
    
        # 언론사선택(우측)
        press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
        # 언론사 선택할 수 있는 클릭 버튼
        press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
        # 언론사 이름들 추출
        press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
        # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
        press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
        # 원하는 언론사가 해당 이름 안에 있는 경우
        # 1) 클릭하고
        # 2) 더이상 언론사분류선택 탐색 중지
        if press_nm in press_slct_btn_dict.keys():
            print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
            press_slct_btn_dict[press_nm].click()
            time.sleep(sleep_sec)
            
            break



################ 뉴스 크롤링 ########################
    print('\n크롤링을 시작합니다.')
    news_dict = {}
    idx = 1
    cur_page = 1

    pbar = tqdm(total=ct ,leave = True)
    
    while idx < ct:

        table = browser.find_element_by_xpath('//ul[@class="list_news"]')
        li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
        area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
        a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
        for n in a_list[:min(len(a_list), ct-idx+1)]:
            n_url = n.get_attribute('href')
            news_dict[idx] = {'title' : n.get_attribute('title'), 
                              'url' : n_url,
                              'text' : crawling_main_text(n_url)}
        
            idx += 1
            pbar.update(1)
        
        if idx < ct:
            cur_page +=1

            pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
            next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

            browser.get(next_page_url)
            time.sleep(sleep_sec)
        else:
            pbar.close()
        
            print('\n브라우저를 종료합니다.\n' + '=' * 100)
            time.sleep(0.7)
            browser.close()
            break

#### 데이터 전처리하기 ###################################################### 

    print('데이터프레임 변환\n')
    news_df = DataFrame(news_dict).T

    folder_path = os.getcwd()
    xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(ct, keyword, date)

    news_df.to_excel(xlsx_file_name)

    print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

    os.startfile(folder_path)

    # print('=' * 100 + '\n결과물의 일부')
    news_df
    
    return news_df

In [62]:
import os
import sys
import requests
import json
import urllib.request
# import pandas as pd
# news_df = pd.read_excel('./news_df.xlsx')

def process_b(news_df):
    global r1
    len(news_df.title)
    client_id = "qj40wl5n0t"
    client_secret = "1xzXEYgwlsbv9RQxd1onUSByiyvtBDPFV9NEY1mv"

    for i in range(len(news_df.title)):
        headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }
        language = "ko" # Language of document (ko, ja )
        model = "news" # Model used for summaries (general, news)
        tone = "2" # Converts the tone of the summarized result. (0, 1, 2, 3)
        summaryCount = "3" # This is the number of sentences for the summarized document.
        url= "https://naveropenapi.apigw.ntruss.com/text-summary/v1/summarize" 
        title= ""
        content = news_df.text[i+1]
        data = {
            "document": {
            "title": title,
            "content" : content
            },
            "option": {
            "language": language,
            "model": model,
            "tone": tone,
            "summaryCount" : summaryCount
            }
        }
        # print(json.dumps(data, indent=4, sort_keys=True))
        response = requests.post(url, data=json.dumps(data), headers=headers)
        rescode = response.status_code
        if(rescode == 200):
            r1=response.text  
            # print(r1)
            
        else:
            print("Error : " + response.text)
        
        r1 = json.loads(r1)
        r1d = json.loads(r1)
        # r1 = json.dumps(r1)
        # r1r1= r1['summary']
        r1r1 = r1d['summary']
        #r1r1= r1.split('연합뉴스')[1]
        r2 = '{}번째 뉴스입니다.{}'.format(i+1, r1r1)
        print(r2)
        li = []
        li.append(r2)
        
        encText = urllib.parse.quote(r1r1)
        data = "speaker=njihun&volume=0&speed=0&pitch=0&format=mp3&text=" + encText;
        url = "https://naveropenapi.apigw.ntruss.com/tts-premium/v1/tts"
        request = urllib.request.Request(url)
        request.add_header("X-NCP-APIGW-API-KEY-ID",client_id)
        request.add_header("X-NCP-APIGW-API-KEY",client_secret)
        response = urllib.request.urlopen(request, data=data.encode('utf-8'))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()
            fn = 'voice{}.mp3'.format(i+1)
            with open('/static/'+fn, 'wb') as f:
                f.write(response_body)
            print(fn)
        else:
            print("Error Code:" + rescode)
            
    
    return li

In [42]:
print(r1r1)

NameError: name 'r1r1' is not defined

t=2022-03-26T02:50:41+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=csess id=e91af4ff2dd0 clientid=c7593c5ba6526ae73929c0466d46863b
t=2022-03-26T02:50:41+0900 lvl=eror msg="session closed, starting reconnect loop" obj=csess id=141abe159d04 err="read tcp 192.168.10.101:54229->3.133.228.214:443: wsarecv: An existing connection was forcibly closed by the remote host."
t=2022-03-26T03:06:21+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=csess id=4a23c60ce730 clientid=c7593c5ba6526ae73929c0466d46863b
t=2022-03-26T03:06:21+0900 lvl=eror msg="session closed, starting reconnect loop" obj=csess id=141abe159d04 err="read EOF from remote peer"
t=2022-03-26T11:11:49+0900 lvl=eror msg="session closed, starting reconnect loop" obj=csess id=141abe159d04 err="read EOF from remote peer"
t=2022-03-26T11:11:50+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=csess id=a4512a66de20 clientid=c7593c5ba6526ae73929c0466d46863b
t=2022-03-26T11:38:17+0900 lvl=eror m

In [20]:
audio1 = open("voice0.mp3", "rb").read()
audio2 = open("voice1.mp3", "rb").read()
audio3 = open("voice2.mp3", "rb").read()
audioJoin = audio1 + audio2 + audio3
audioFinal = open("./templates/audioFinal.mp3", "wb").write(audioJoin)

In [24]:
!mkdir static

In [25]:
from fastapi.staticfiles import StaticFiles
app.mount("/static", StaticFiles(directory="static"), name="static")

In [38]:
app=FastAPI()
templates = Jinja2Templates(directory="templates")

@app.get('/', response_class=HTMLResponse)
async def read_topic(request: Request):
  return templates.TemplateResponse("inputnews.html", {"request": request})

@app.post('/topic', response_class=HTMLResponse)
async def get_topic(request: Request, keyword: str = Form(...), ct: int = Form(...), result_news: str=Form(...) ):
    news_df = process_a(keyword, ct)
    result_news = process_b(news_df)
    #fn = get_mp3()

    return templates.TemplateResponse("resultnews.html", {"request": request, 'keyword': keyword, 'ct':ct, 'result_news': result_news})

# @app.get("/topic", response_class=HTMLResponse)
# def voice(request: Request):
#     out = []
#     for filename in os.listdir("static/"):
#         out.append({
#             "name": filename.split(".")[0],
#             "path": "/static/" + filename
#         })
#     return templates.TemplateResponse('resultnews.html', {'request': request, 'fn': fn})


In [65]:

result = process_b(process_a('고양이', 2))

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : MBC


브라우저를 실행시킵니다(자동 제어)



C:\Users\class\AppData\Local\Temp/ipykernel_18216/2121326648.py:79: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)


설정한 언론사를 선택합니다.



C:\Users\class\AppData\Local\Temp/ipykernel_18216/2121326648.py:89: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\class\AppData\Local\Temp/ipykernel_18216/2121326648.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')


<방송/통신> 카테고리에서 <MBC>를 찾았으므로 탐색을 종료합니다

크롤링을 시작합니다.


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]C:\Users\class\AppData\Local\Temp/ipykernel_18216/2121326648.py:146: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  table = browser.find_element_by_xpath('//ul[@class="list_news"]')
100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.98it/s]



브라우저를 종료합니다.
데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\class\OneDrive\문서\네이버뉴스_본문_2개_고양이_2022-03-28_10시22분.xlsx

Error : {"status":400,"error":{"errorCode":"E003","message":"Text quota Exceeded"}}


NameError: name 'r1' is not defined

In [59]:
df = process_a('반려견',2)

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : MBC


브라우저를 실행시킵니다(자동 제어)



C:\Users\class\AppData\Local\Temp/ipykernel_18216/2079200753.py:79: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(chrome_path)


설정한 언론사를 선택합니다.



C:\Users\class\AppData\Local\Temp/ipykernel_18216/2079200753.py:89: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
C:\Users\class\AppData\Local\Temp/ipykernel_18216/2079200753.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00AC9943+2595139]
	Ordinal0 [0x00A5C9F1+2148849]
	Ordinal0 [0x009543F0+1065968]
	Ordinal0 [0x00981F17+1253143]
	Ordinal0 [0x00977AB3+1211059]
	Ordinal0 [0x0099B9EC+1358316]
	Ordinal0 [0x00977474+1209460]
	Ordinal0 [0x0099BC04+1358852]
	Ordinal0 [0x009ABAF2+1424114]
	Ordinal0 [0x0099B806+1357830]
	Ordinal0 [0x00976086+1204358]
	Ordinal0 [0x00976F96+1208214]
	GetHandleVerifier [0x00C6B232+1658114]
	GetHandleVerifier [0x00D2312C+2411516]
	GetHandleVerifier [0x00B5F261+560433]
	GetHandleVerifier [0x00B5E366+556598]
	Ordinal0 [0x00A6286B+2173035]
	Ordinal0 [0x00A675F8+2192888]
	Ordinal0 [0x00A676E5+2193125]
	Ordinal0 [0x00A711FC+2232828]
	BaseThreadInitThunk [0x75F26739+25]
	RtlGetFullPathName_UEx [0x77558E7F+1215]
	RtlGetFullPathName_UEx [0x77558E4D+1165]


In [13]:
!ngrok authtoken 26P5PTLiceT3jUQpHS9gh4yOvFV_2P7DFVLKYpsARbZfaWmGR

Authtoken saved to configuration file: C:\Users\class/.ngrok2/ngrok.yml


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print ('Public URL:', ngrok_tunnel.public_url) 
nest_asyncio.apply()
uvicorn.run(app, host='0.0.0.0', port=8000)

### 파일을 합쳐서 새로운 파일로. 
audio1 = open("voice0.mp3", "rb").read()
audio2 = open("voice1.mp3", "rb").read()
audio3 = open("voice2.mp3", "rb").read()
audioJoin = audio1 + audio2 + audio3
audioFinal = open("./templates/audioFinal.mp3", "wb").write(audioJoin)

In [ ]:
@app.get("/")
def main():
    def iterfile():  # 
        with open(some_file_path, mode="rb") as file_like:  # 
            yield from file_like  # 

    return StreamingResponse(iterfile(), media_type="video/mp3")

In [30]:
def get_mp3():
    p_list = find_file()  
    li = p_list.reverse()
    fname = li.pop()
    return fname



In [29]:
## 현재폴더 mp3 확장자 파일 찾아줌. 
def find_file():
    fileDir = './static/'
    #fileDir = os.getcwd()
    fileExt = r".mp3"
    list = [os.path.join(fileDir, _) for _ in os.listdir(fileDir) if _.endswith(fileExt)]
    path_list = []
    for path in list:
        path = path.replace('\\', '/')
        path_list.append(path)
    return path_list 

# ## 소리 재생 , 0누르면 종료 
# def play_sound(path_list):
#     for path in path_list :
#         playsound.playsound(path, False)
#         k = int(input('Press 0 if you want to stop'))
#         if k == 0: 
#             playsound(None)

    

In [8]:
import playsound 


In [12]:
from typing import Optional
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

@app.get('/')
async def home():
  return 'Hello World'

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, host = '0.0.0.0', port=8000)

t=2022-03-25T14:18:41+0900 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040
t=2022-03-25T14:18:42+0900 lvl=eror msg="failed to auth" obj=tunnels.session err="Your account is limited to 1 simultaneous ngrok client session.\nActive ngrok client sessions in region 'us':\n  - ts_26rVhFF43U5zrSB9THJpJiKDnX7 (35.231.139.213)\r\n\r\nERR_NGROK_108\r\n"
t=2022-03-25T14:18:42+0900 lvl=eror msg="failed to reconnect session" obj=csess id=fc6addbdc3da err="Your account is limited to 1 simultaneous ngrok client session.\nActive ngrok client sessions in region 'us':\n  - ts_26rVhFF43U5zrSB9THJpJiKDnX7 (35.231.139.213)\r\n\r\nERR_NGROK_108\r\n"
t=2022-03-25T14:18:42+0900 lvl=eror msg="session closing" obj=tunnels.session err="Your account is limited to 1 simultaneous ngrok client session.\nActive ngrok client sessions in region 'us':\n  - ts_26rVhFF43U5zrSB9THJpJiKDnX7 (35.231.139.213)\r\n\r\nERR_NGROK_108\r\n"
t=2022-03-25T14:18:42+0900 lvl=eror msg="term

PyngrokNgrokError: The ngrok process errored on start: Your account is limited to 1 simultaneous ngrok client session.\nActive ngrok client sessions in region 'us':\n  - ts_26rVhFF43U5zrSB9THJpJiKDnX7 (35.231.139.213)\r\n\r\nERR_NGROK_108\r\n.